In [2]:
import time
import requests
import numpy as np
import pandas as pd
from scrapy.http import TextResponse
import re
import scrapy
from scrapy.crawler import CrawlerProcess
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import yfinance as yf
import quandl

## 2 [ Yahoo ]

In [3]:
tsla_df = yf.download('TSLA', 
                      start='2018-07-26', 
                      end='2020-07-26',
                      progress=False)

In [4]:
tsla_df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-07-25,296.739990,309.619995,294.500000,308.739990,308.739990,7075400
2018-07-26,304.850006,310.700012,303.640015,306.649994,306.649994,4630500
2018-07-27,307.250000,307.690002,295.339996,297.179993,297.179993,5703300
2018-07-30,295.899994,296.100006,286.130005,290.170013,290.170013,6814100
2018-07-31,292.250000,298.320007,289.070007,298.140015,298.140015,5076900
...,...,...,...,...,...,...
2020-07-20,1519.010010,1650.000000,1488.000000,1643.000000,1643.000000,17121400
2020-07-21,1639.930054,1675.000000,1558.000000,1568.359985,1568.359985,16157300
2020-07-22,1599.000000,1626.420044,1562.000000,1592.329956,1592.329956,14161100


In [5]:
tsla_df['Open'].pct_change().mean()

0.004210699741383978

In [6]:
tsla_df['High'].pct_change().median() - tsla_df['Low'].pct_change().median()

-0.0031558615890963093

## 2 [ Quandl ]

In [7]:
data=quandl.get("WIKI/TSLA",authtoken="d4U5nvsQhoDy6RgUAgt7",
                  collapse = 'daily',start_date="2015-07-26",end_date="2017-07-26")

In [10]:
data

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2015-07-27,262.43,264.4321,250.7900,253.01,4694192.0,0.0,1.0,262.43,264.4321,250.7900,253.01,4694192.0
2015-07-28,255.75,265.4000,251.8373,264.82,3895808.0,0.0,1.0,255.75,265.4000,251.8373,264.82,3895808.0
2015-07-29,264.27,267.8900,262.0000,263.82,2790095.0,0.0,1.0,264.27,267.8900,262.0000,263.82,2790095.0
2015-07-30,262.69,266.9400,262.1100,266.79,2034560.0,0.0,1.0,262.69,266.9400,262.1100,266.79,2034560.0
2015-07-31,267.60,269.3599,265.1230,266.15,2222552.0,0.0,1.0,267.60,269.3599,265.1230,266.15,2222552.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2017-07-20,326.90,330.2200,324.2000,329.92,5098707.0,0.0,1.0,326.90,330.2200,324.2000,329.92,5098707.0
2017-07-21,329.46,331.2575,325.8000,328.40,4826383.0,0.0,1.0,329.46,331.2575,325.8000,328.40,4826383.0
2017-07-24,330.24,343.3990,330.0100,342.52,8552012.0,0.0,1.0,330.24,343.3990,330.0100,342.52,8552012.0


In [8]:
data['Open'].pct_change().mean()

0.0008780826439791697

In [9]:
data['High'].pct_change().median() - data['Low'].pct_change().median()

-0.0013851194579926895

In [ ]:
#https://db.nomics.world/IMF/DOT?dimensions=%7B%22COUNTERPART_AREA%22%3A%5B%22W00%22%5D%7D&q=armenia%20imports